# BEE 4750 Homework 5: Mixed Integer and Stochastic Programming

**Name**:

**ID**:

> **Due Date**
>
> Thursday, 12/05/24, 9:00pm

## Overview

### Instructions

-   In Problem 1, you will use mixed integer programming to solve a
    waste load allocation problem.

### Load Environment

The following code loads the environment and makes sure all needed
packages are installed. This should be at the start of most Julia
scripts.

In [2]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

  Activating project at `~/Local/Coding Projects/BEE4570/hw5-super-cool-team-name`


In [3]:
using JuMP
using HiGHS
using DataFrames
using GraphRecipes
using Plots
using Measures
using MarkdownTables

## Problems (Total: 30 Points)

### Problem 1 (30 points)

Three cities are developing a coordinated municipal solid waste (MSW)
disposal plan. Three disposal alternatives are being considered: a
landfill (LF), a materials recycling facility (MRF), and a
waste-to-energy facility (WTE). The capacities of these facilities and
the fees for operation and disposal are provided below.

-   **LF**: Capacity 200 Mg, fixed cost \$2000/day, tipping cost
    \$50/Mg;
-   **MRF**: Capacity 350 Mg, fixed cost \$1500/day, tipping cost
    \$7/Mg, recycling cost \$40/Mg recycled;
-   **WTE**: Capacity 210 Mg, fixed cost \$2500/day, tipping cost
    \$60/Mg;

The MRF recycling rate is 40%, and the ash fraction of non-recycled
waste is 16% and of recycled waste is 14%. Transportation costs are
\$1.5/Mg-km, and the relative distances between the cities and
facilities are provided in the table below.

| **City/Facility** | **Landfill (km)** | **MRF (km)** | **WTE (km)** |
|:-----------------:|:-----------------:|:------------:|:------------:|
|         1         |         5         |      30      |      15      |
|         2         |        15         |      25      |      10      |
|         3         |        13         |      45      |      20      |
|        LF         |        \-         |      32      |      18      |
|        MRF        |        32         |      \-      |      15      |
|        WTE        |        18         |      15      |      \-      |

The fixed costs associated with the disposal options are incurred only
if the particular disposal option is implemented. The three cities
produce 100, 90, and 120 Mg/day of solid waste, respectively, with the
composition provided in the table below.

**Reminder**: Use `round(x; digits=n)` to report values to the
appropriate precision!

**In this problem**:

-   Formulate the waste load allocation problem and implement it in
    `JuMP`.
-   Draw a diagram showing the flows of waste between the cities and the
    facilities. Which facilities (if any) will not be used? Does this
    solution make sense?

**Problem Formulation**
This will be a mixed integer linear program:

Numbering treatment options: 1 - LF, 2 - MRF, 3 - WTE

Notation: 
$F_{j}$ - Fixed costs for each option j

$V_{j}$ - Variable (tipping) costs for each option j

$C_r$ - Recycling cost

$C_t$ - Transport

$D_{i,j}$ - Distance between city i and option j

$D_{r,j,k}$ - Distance for residual travel between option k and option k

$R_{r}$ - Recycling Rate

$A_{r}$ - Ash fraction of recycled material (The way we are interpreting this is ash produced during recycling, as you wouldn't burn recycled goods for energy)

$A_{n}$ - Ash fraction of non-recycled material (fraction of WTE that must be diverted to landfill)

$P_i$ - Waste produced by city i

$C_j$ - Capacity of plant j


Decision Variables:

$W_{i,j}$ - Amount to send to each city i to option j

$R_{j,k}$ - Amount of residual to send from option j to option k, where (j,k) can be (2,1), (2,3), or (3,1)

$Y_{j}$ - Whether or not to use each option j, binary variable

Objective Function: cost of running each plant that is running, tipping + transport cost for each flow from city to plant, cost of recycling, and transportation + tipping costs for all residual flows

Min $ \sum_{j} [Y_j F_j + \sum_{i} (C_t D_{i,j} + V_j) W_{i,j}] + C_r R_r\sum_{i} W_{i,j} + R_{2,1} (D_{r,2,1} C_t + V_1) + R_{3,1} (D_{r,3,1} C_t + V_1) + R_{2,3} (D_{r,2,3} C_t + V_3)$

Subject to:

$\sum_{j}W_{i,j} +\sum_{k}R_{j,k} \leq C_j \forall j$ (Capacity limitation)

$\sum_{i}W_{i,j} = P_j \forall i$     (Cities must dispose all their waste)

$R_{2,1} \geq R_rA_r \times [\sum_{i} W_{i,2}]$ (Ash produced by recycling must be disposed in landfill)

$R_{3,1} = A_n ([\sum_{j} W_{3,j}] + R_{2,3})$ {Ash Produced by WTE must be landfilled}

$R_{2,1} + R_{2,3} = (1-R_r) \times [ \sum_{i} W_{i,2}] + R_rA_r \times [\sum_{i} W_{i,2}]$ (Everything that does not get recycled, and ash recycling waste, must be otherwise disposed)

for all i:

$Y_i = 1$ if $\sum_{j} W_{i,j} + \sum_{k} R_{k,i} \geq 0$

$Y_i = 0$ otherwise

In [ ]:
model = Model(HiGHS.Optimizer)
@variable(model, 0 <= W[i=1:6,j=1:3])
@variable(model, 0 <= Y[1:3] <= 1, Int)
@variable(model, 0 <= R21)
@variable(model, 0 <= R31)
@variable(model, 0 <= R23)

# Constants
Ct = 1.5        # transport cost
Cr = 40         # recycle cost
Rr = 0.4        # recycle rate
Ar = 0.14       # ash frac - recycled
An = 0.16       # ash frac - non recycled

# Distances-- order is j, i in indexing
dist = DataFrame(LF=[5, 15, 13, 0, 32, 18], MRF=[30,25,45,32,0,15], WTE=[15,10,20,18,15,0])
F = [2000, 1500, 2500]  # Fixed costs
V = [50, 7, 60]         # Variable costs
C = [200, 350, 210]     # Capacities
P = [100, 90, 120]      # Waste production

@objective(model, Min, 
    sum((Y[j]*F[j] + sum((Ct*dist[i,j]+V[j])*W[i,j] for i=1:3)) for j=1:3) + 
    Cr*Rr*sum(W[i,2] for i=1:3) + 
    R21*(dist[5,1]*Ct + V[1]) +
    R31*(dist[6,1]*Ct + V[1]) +
    R23*(dist[5,3]*Ct + V[3])
    )

@constraint(model, capacityLF, sum(W[i,1] for i=1:3) + R21 + R31 <= C[1])
@constraint(model, capacityMRF, sum(W[i,2] for i=1:3) <= C[2])
@constraint(model, capacityWTE, sum(W[i,3] for i=1:3) + R23 <= C[3])

@constraint(model, disposal1, sum(W[1,j] for j=1:3) == P[1])
@constraint(model, disposal2, sum(W[2,j] for j=1:3) == P[2])
@constraint(model, disposal3, sum(W[3,j] for j=1:3) == P[3])

@constraint(model, ashRtoLF, Rr * Ar * sum(W[i,2] for i=1:3) == R21)
@constraint(model, ashWTEtoLF, An * (sum(W[i,3] for i=1:3)+R23) == R23)
@constraint(model, moredispo, (1-Rr)*sum(W[i,2] for i=1:3) + Rr*Ar * sum(W[i,2] for i=1:3) == R21 + R23)

M=1000000
@constraint(model, sum(W[i,1] for i=1:3) + R21 + R31 <= M*Y[1])
@constraint(model, sum(W[i,1] for i=1:3) + R21 + R31 >= -M*Y[1])

@constraint(model, sum(W[i,2] for i=1:3) <= M*Y[2])
@constraint(model, sum(W[i,2] for i=1:3) >= -M*Y[2])

@constraint(model, sum(W[i,3] for i=1:3) + R23 <= M*Y[3])
@constraint(model, sum(W[i,3] for i=1:3) + R23 >= -M*Y[3])

optimize!(model)

@show value.(R21)
@show value.(R31)
@show value.(R23)
@show value.(W)


Running HiGHS 1.8.1 (git hash: 4a7f24ac6): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [6e-02, 1e+06]
  Cost   [6e+01, 2e+03]
  Bound  [1e+00, 1e+00]
  RHS    [9e+01, 4e+02]
Presolving model
11 rows, 14 cols, 44 nonzeros  0s
9 rows, 12 cols, 39 nonzeros  0s
6 rows, 9 cols, 24 nonzeros  0s

Solving MIP model with:
   6 rows
   9 cols (0 binary, 0 integer, 0 implied int., 9 continuous)
   24 nonzeros
MIP-Timing:     0.00018 - starting analytic centre calculation

Src: B => Branching; C => Central rounding; F => Feasibility pump; H => Heuristic; L => Sub-MIP;
     P => Empty MIP; R => Randomized rounding; S => Solve LP; T => Evaluate node; U => Unbounded;
     z => Trivial zero; l => Trivial lower; u => Trivial upper; p => Trivial point

        Nodes      |    B&B Tree     |            Objective Bounds              |  Dynamic Constraints |       Work      
Src  Proc. InQueue |  Leaves   Expl. | BestBound       BestSol              Gap |   Cuts   InLp Con

6×3 Matrix{Float64}:
 100.0      0.0      0.0
   0.0     26.8686  63.1314
  98.4954   0.0     21.5046
   0.0      0.0      0.0
   0.0      0.0      0.0
   0.0      0.0      0.0

## References

List any external references consulted, including classmates.